In [1]:
# 公式からとってくる。
# レースリスト、レースの参加メンバー、直前のオッズリストをESに突っ込む
# https://www.boatrace.jp/owpc/pc/race/index?hd=%s
# http://odds.kyotei24.jp/

from bs4 import BeautifulSoup
import json
import urllib.request as u
import time
import re
import elasticsearch as es

import unicodedata

In [2]:
with open('../config/def-place.json','r',encoding='utf8') as defPlace:
    placeDict=json.load(defPlace)
print(placeDict[0]['placeId'],placeDict[0]['placeName'])
#for i in range(len(placeDict)):
#    print("|"+placeDict[i]['placeId']+"|"+placeDict[i]['placeName']+"|")

01 桐生


In [3]:
## 使わない。
def getRaceList(ymd):
    html = u.urlopen("http://odds.kyotei24.jp/index-%s.html" % ymd)
    time.sleep(1)
    soup = BeautifulSoup(html, "html.parser")
    for block in soup.select(".table.table-bordered"):
        idx = 0
        recs = block.select("tr")
        if len(recs) >= 1:
            kaijou = ''
            name = ''
            for rec in recs:
                tds = rec.select('td')
                if len(tds) > 1:
                    no = 0
                    for h in rec.select('a'):
                        no = h.string
                        kaijou=h['href'][14:16]
                        # todo Gradeが取れてない
                        gradepattern=r"【(.*)】"
                        grade=re.match(gradepattern , name)
                        detailUrl=h['href']
                        raceId = ymd+'-'+kaijou+'-'+no.zfill(2)
                        print(raceId,ymd,no,name,grade,kaijou,detailUrl)
                else:
                    head = tds.pop()
                    kaijou = head.b.string
                    name = head.contents[1].string


In [4]:
def getRaceListKoushiki(ymd):
    html = u.urlopen("https://www.boatrace.jp/owpc/pc/race/index?hd=%s" % ymd)
    time.sleep(1)
    soup = BeautifulSoup(html,"html.parser")
    for block in soup.select(".table1"):
        recs=block.select("tr")
        for rec in recs:
            tds=rec.select("td")
            if len(tds)>0:
                tmp=tds[0]
                #placePattern=r'tpl=(\d\d)'
                placePattern=r'tv(\d*)'
                placeId=re.search(placePattern,str(tmp)).group(1)
                #print(placeId)
                
                tmp=tds[2]
                gradePattern=r'class=\"is-(.*?)\"'
                grade=re.search(gradePattern,str(tmp)).group(1)
                #print(grade)
                
                rname=tds[4].string
                #print(rname)
                
                for i in range(1,12+1):
                    raceId=ymd+'-'+str(placeId).zfill(2)+'-'+str(i).zfill(2)
                    raceDate=ymd
                    raceNumber=i
                    raceName=rname
                    raceGrade=grade
                    racePlaceId=str(placeId).zfill(2)
                    
                    yield raceId,raceDate,raceNumber,raceName,raceGrade,racePlaceId

ymd = "20190321"
for ret in getRaceListKoushiki(ymd):
    print(ret)

('20190321-01-01', '20190321', 1, 'みどり市発足１３周年記念・市議会議長杯', 'ippan', '01')
('20190321-01-02', '20190321', 2, 'みどり市発足１３周年記念・市議会議長杯', 'ippan', '01')
('20190321-01-03', '20190321', 3, 'みどり市発足１３周年記念・市議会議長杯', 'ippan', '01')
('20190321-01-04', '20190321', 4, 'みどり市発足１３周年記念・市議会議長杯', 'ippan', '01')
('20190321-01-05', '20190321', 5, 'みどり市発足１３周年記念・市議会議長杯', 'ippan', '01')
('20190321-01-06', '20190321', 6, 'みどり市発足１３周年記念・市議会議長杯', 'ippan', '01')
('20190321-01-07', '20190321', 7, 'みどり市発足１３周年記念・市議会議長杯', 'ippan', '01')
('20190321-01-08', '20190321', 8, 'みどり市発足１３周年記念・市議会議長杯', 'ippan', '01')
('20190321-01-09', '20190321', 9, 'みどり市発足１３周年記念・市議会議長杯', 'ippan', '01')
('20190321-01-10', '20190321', 10, 'みどり市発足１３周年記念・市議会議長杯', 'ippan', '01')
('20190321-01-11', '20190321', 11, 'みどり市発足１３周年記念・市議会議長杯', 'ippan', '01')
('20190321-01-12', '20190321', 12, 'みどり市発足１３周年記念・市議会議長杯', 'ippan', '01')
('20190321-02-01', '20190321', 1, '第５４回ボートレースクラシック', 'SGa', '02')
('20190321-02-02', '20190321', 2, '第５４回ボートレースクラシック', 'SGa', '02')
(

In [5]:
def getRaceMemberKoushiki(raceId):
    #html = u.urlopen("https://www.boatrace.jp/owpc/pc/race/index?hd=%s" % ymd)
    raceDate=raceId[0:8]
    raceNumber=int(raceId[12:14])
    placeId=raceId[9:11]
    print(raceDate,raceNumber,placeId)
    html = u.urlopen("https://www.boatrace.jp/owpc/pc/race/racelist?rno=%d&jcd=%s&hd=%s" %(raceNumber,placeId,raceDate) )
    time.sleep(1)
    soup = BeautifulSoup(html,"html.parser")
    for block in soup.select(".is-fs12"):
        #print(block)
        recs=block.select("tr")
        rec=recs[0]
        tds=rec.select("td")
        lane=unicodedata.normalize('NFKC', tds[0].string)        
        tobanId=((tds[2].select(".is-fs11"))[0].contents[0].strip())[0:4]
        rank=(tds[2].select("span"))[0].string
        #print("lane:",lane)
        #print("tobanId:",tobanId)
        #print("rank:",rank)
        #print(tds[2]) ※　他に取りたい項目があればここからとってくる
        
        yield lane,tobanId,rank
        
raceId="20190321-01-01"
for ret in getRaceMemberKoushiki(raceId):
    print(ret)


20190321 1 01
('1', '3812', 'B1')
('2', '3123', 'B1')
('3', '3261', 'B1')
('4', '3290', 'A1')
('5', '5063', 'B2')
('6', '3673', 'B1')


In [6]:
def getOldOddsKoushiki3rentan(raceId):
    raceDate=raceId[0:8]
    raceNumber=int(raceId[12:14])
    placeId=raceId[9:11]
    print(raceDate,raceNumber,placeId)
    html = u.urlopen("https://www.boatrace.jp/owpc/pc/race/odds3t?rno=%d&jcd=%s&hd=%s" %(raceNumber,placeId,raceDate) )
    time.sleep(1)
    soup = BeautifulSoup(html,"html.parser")
    
    # 3連単の組み合わせリストを辞書順に作成
    sanrentanList=[]
    for i in range(1,6+1):
        for j in range(1,6+1):
            for k in range(1,6+1):
                if (i==j or i==k or j==k):
                    continue
                else:
                    sanrentanList.append(str(i)+"-"+str(j)+"-"+str(k))
    sorted(sanrentanList)
    
    sanrentanOddsList=[]
    for oddspoint in soup.select(".oddsPoint"):
        sanrentanOddsList.append(oddspoint.string)
    
    cnt=0
    for i in range(6):
        for j in range(20):
            yield sanrentanList[i*20+j],sanrentanOddsList[i+j*6]
    
raceId="20190321-01-01"
for ret in getOldOddsKoushiki3rentan(raceId):
    print(ret)


20190321 1 01
('1-2-3', '47.0')
('1-2-4', '27.2')
('1-2-5', '238.8')
('1-2-6', '98.8')
('1-3-2', '45.4')
('1-3-4', '19.3')
('1-3-5', '210.3')
('1-3-6', '50.8')
('1-4-2', '15.6')
('1-4-3', '13.1')
('1-4-5', '130.1')
('1-4-6', '25.3')
('1-5-2', '525.1')
('1-5-3', '433.0')
('1-5-4', '303.4')
('1-5-6', '679.3')
('1-6-2', '146.6')
('1-6-3', '101.0')
('1-6-4', '74.4')
('1-6-5', '416.0')
('2-1-3', '84.3')
('2-1-4', '48.7')
('2-1-5', '336.6')
('2-1-6', '148.6')
('2-3-1', '134.3')
('2-3-4', '85.2')
('2-3-5', '787.7')
('2-3-6', '199.6')
('2-4-1', '37.8')
('2-4-3', '51.7')
('2-4-5', '317.8')
('2-4-6', '87.2')
('2-5-1', '841.5')
('2-5-3', '1346')
('2-5-4', '1000')
('2-5-6', '1806')
('2-6-1', '295.0')
('2-6-3', '244.3')
('2-6-4', '206.8')
('2-6-5', '1322')
('3-1-2', '91.4')
('3-1-4', '35.8')
('3-1-5', '252.7')
('3-1-6', '76.8')
('3-2-1', '125.0')
('3-2-4', '83.1')
('3-2-5', '597.1')
('3-2-6', '143.7')
('3-4-1', '34.9')
('3-4-2', '57.5')
('3-4-5', '359.4')
('3-4-6', '53.6')
('3-5-1', '718.9')
('3-5-

In [7]:
def getOldOddsKoushiki3renfuku(raceId):
    raceDate=raceId[0:8]
    raceNumber=int(raceId[12:14])
    placeId=raceId[9:11]
    print(raceDate,raceNumber,placeId)
    html = u.urlopen("https://www.boatrace.jp/owpc/pc/race/odds3f?rno=%d&jcd=%s&hd=%s" %(raceNumber,placeId,raceDate) )
    time.sleep(1)
    soup = BeautifulSoup(html,"html.parser")
    
    # 3連複の組み合わせリストをサイトからの取得可能順に作成
    sanrenfukuList=[]
    for k in range(3,6+1):
        sanrenfukuList.append(str(1)+"-"+str(2)+"-"+str(k))
    for k in range(4,6+1):
        for i in range(1,2+1):
            sanrenfukuList.append(str(i)+"-"+str(3)+"-"+str(k))
    for k in range(5,6+1):
        for i in range(1,3+1):
            sanrenfukuList.append(str(i)+"-"+str(4)+"-"+str(k))
    for i in range(1,4+1):
        sanrenfukuList.append(str(i)+"-"+str(5)+"-"+str(6))
    
    #print(sanrenfukuList)
    sanrenfukuOddsList=[]
    for oddspoint in soup.select(".oddsPoint"):
        sanrenfukuOddsList.append(oddspoint.string)
    
    for i in range(len(sanrenfukuList)):
        yield sanrenfukuList[i],sanrenfukuOddsList[i]
    
raceId="20190321-01-01"
for ret in getOldOddsKoushiki3renfuku(raceId):
    print(ret)



20190321 1 01
('1-2-3', '15.1')
('1-2-4', '2.9')
('1-2-5', '85.2')
('1-2-6', '37.9')
('1-3-4', '2.6')
('2-3-4', '9.5')
('1-3-5', '93.7')
('2-3-5', '156.2')
('1-3-6', '17.4')
('2-3-6', '42.6')
('1-4-5', '46.0')
('2-4-5', '87.8')
('3-4-5', '117.1')
('1-4-6', '9.7')
('2-4-6', '16.2')
('3-4-6', '12.7')
('1-5-6', '127.8')
('2-5-6', '200.8')
('3-5-6', '147.9')
('4-5-6', '140.5')


In [8]:
def getOldOddsKoushiki2rentan(raceId):
    raceDate=raceId[0:8]
    raceNumber=int(raceId[12:14])
    placeId=raceId[9:11]
    print(raceDate,raceNumber,placeId)
    html = u.urlopen("https://www.boatrace.jp/owpc/pc/race/odds2tf?rno=%d&jcd=%s&hd=%s" %(raceNumber,placeId,raceDate) )
    time.sleep(1)
    soup = BeautifulSoup(html,"html.parser")
    
    block =(soup.select(".is-p3-0"))[0]
    tmpcnt1=1
    for trs in block.select("tr"):
        tmpcnt2=0
        tmp1stlane=1
        for tds in trs.select("td"):
            if tmpcnt2%2==0:
                cs=tds.string
            else:
                odds=tds.string
                yield str(tmp1stlane)+"-"+str(cs),odds
                tmp1stlane+=1
            tmpcnt2+=1
        tmpcnt1+=1
        
raceId="20190321-01-01"
for ret in getOldOddsKoushiki2rentan(raceId):
    print(ret)



20190321 1 01
('1-2', '14.0')
('2-1', '28.6')
('3-1', '13.1')
('4-1', '4.5')
('5-1', '178.5')
('6-1', '82.0')
('1-3', '9.0')
('2-3', '42.1')
('3-2', '29.7')
('4-2', '9.3')
('5-2', '505.7')
('6-2', '112.3')
('1-4', '5.7')
('2-4', '14.0')
('3-4', '16.7')
('4-3', '9.2')
('5-3', '379.3')
('6-3', '82.0')
('1-5', '86.7')
('2-5', '202.3')
('3-5', '131.9')
('4-5', '67.4')
('5-4', '151.7')
('6-4', '52.3')
('1-6', '29.4')
('2-6', '89.2')
('3-6', '21.6')
('4-6', '25.7')
('5-6', '216.7')
('6-5', '202.3')


In [20]:
def getOldOddsKoushiki2renfuku(raceId):
    raceDate=raceId[0:8]
    raceNumber=int(raceId[12:14])
    placeId=raceId[9:11]
    print(raceDate,raceNumber,placeId)
    html = u.urlopen("https://www.boatrace.jp/owpc/pc/race/odds2tf?rno=%d&jcd=%s&hd=%s" %(raceNumber,placeId,raceDate) )
    time.sleep(1)
    soup = BeautifulSoup(html,"html.parser")
    
    block =(soup.select(".is-p3-0"))[1]
    tmpcnt1=1
    for trs in block.select("tr"):
        tmpcnt2=0
        tmp1stlane=1
        for tds in trs.select("td"):
            if tmpcnt2%2==0:
                cs=tds.string
            elif tds['class'][0] == "oddsPoint":
                odds=tds.string
                yield str(tmp1stlane)+"-"+str(cs),odds
                tmp1stlane+=1
            tmpcnt2+=1
        tmpcnt1+=1
        
raceId="20190321-01-01"
for ret in getOldOddsKoushiki2renfuku(raceId):
    print(ret)




20190321 1 01
('1-2', '9.0')
('1-3', '5.3')
('2-3', '24.4')
('1-4', '2.1')
('2-4', '7.4')
('3-4', '6.6')
('1-5', '105.8')
('2-5', '90.7')
('3-5', '105.8')
('4-5', '63.5')
('1-6', '22.6')
('2-6', '39.7')
('3-6', '17.1')
('4-6', '15.1')
('5-6', '317.6')


In [29]:
def getOldOddsKoushikiTansho(raceId):
    raceDate=raceId[0:8]
    raceNumber=int(raceId[12:14])
    placeId=raceId[9:11]
    print(raceDate,raceNumber,placeId)
    html = u.urlopen("https://www.boatrace.jp/owpc/pc/race/oddstf?rno=%d&jcd=%s&hd=%s" %(raceNumber,placeId,raceDate) )
    time.sleep(1)
    soup = BeautifulSoup(html,"html.parser")
    
    block =((soup.select(".grid_unit"))[0].select("table")[0]).select(".oddsPoint")
    i=1
    for tds in block:
        yield i,tds.string
        i+=1
    
raceId="20190321-01-01"
for ret in getOldOddsKoushikiTansho(raceId):
    print(ret)


20190321 1 01
(1, '2.8')
(2, '20.1')
(3, '8.3')
(4, '1.2')
(5, '0.0')
(6, '33.5')


In [31]:
def getOldOddsKoushikiFukusho(raceId):
    raceDate=raceId[0:8]
    raceNumber=int(raceId[12:14])
    placeId=raceId[9:11]
    print(raceDate,raceNumber,placeId)
    html = u.urlopen("https://www.boatrace.jp/owpc/pc/race/oddstf?rno=%d&jcd=%s&hd=%s" %(raceNumber,placeId,raceDate) )
    time.sleep(1)
    soup = BeautifulSoup(html,"html.parser")
    
    block =((soup.select(".grid_unit"))[1].select("table")[0]).select(".oddsPoint")
    i=1
    for tds in block:
        yield i,tds.string
        i+=1
    
raceId="20190321-01-01"
for ret in getOldOddsKoushikiFukusho(raceId):
    print(ret)


20190321 1 01
(1, '1.3-3.3')
(2, '2.5-7.2')
(3, '1.4-3.6')
(4, '1.1-1.8')
(5, '0.0-0.0')
(6, '3.6-10.8')
